In [16]:
import catanatron_gym.envs.catanatron_env as catan_env 
# from catanatron_experimental.machine_learning.palyers.value import ValueFunctionPlayer
import catanatron_experimental.machine_learning.players.value as value
from catanatron.models.player import Color

In [7]:
# import random
# import gymnasium as gym


# env = gym.make("catanatron_gym:catanatron-v1")
# observation, info = env.reset()
# for _ in range(1000):
#     # your agent here (this takes random actions)
#     action = random.choice(env.unwrapped.get_valid_actions())
#     print(catan_env.ACTIONS_ARRAY[action])
#     observation, reward, terminated, truncated, info = env.step(action)
#     done = terminated or truncated
#     if done:
#         observation, info = env.reset()
# env.close()


In [18]:

env = catan_env.CatanatronEnv(config={
    'enemies': [value.ValueFunctionPlayer(Color.RED)]
})

In [25]:
import random

observation, info = env.reset()
for _ in range(1000):
    # your agent here (this takes random actions)
    action = random.choice(env.unwrapped.get_valid_actions())


    
    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    if done:
        print(env.game.winning_color())
        observation, info = env.reset()
        # print(env.game)
env.close()

Color.RED
Color.RED
Color.RED
Color.RED
Color.RED
Color.RED
Color.RED
Color.RED
Color.RED
Color.RED
Color.RED
Color.RED
